In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [55]:
import os

move_filename = '5365862'
moves_filepath = os.path.join('outputs', 'moves.json')

In [57]:
!ffmpeg -i inputs/videos/5365862.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/5365862.wav

ffmpeg version 2025-04-23-git-25b0a8e295-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev3, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx

In [59]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

array([[ 0.28,  4.  ],
       [ 0.76,  1.  ],
       [ 1.26,  2.  ],
       [ 1.76,  3.  ],
       [ 2.26,  4.  ],
       [ 2.76,  1.  ],
       [ 3.22,  2.  ],
       [ 3.7 ,  3.  ],
       [ 4.2 ,  4.  ],
       [ 4.68,  1.  ],
       [ 5.16,  2.  ],
       [ 5.66,  3.  ],
       [ 6.16,  4.  ],
       [ 6.64,  1.  ],
       [ 7.14,  2.  ],
       [ 7.64,  3.  ],
       [ 8.12,  4.  ],
       [ 8.62,  1.  ],
       [ 9.1 ,  2.  ],
       [ 9.6 ,  3.  ],
       [10.1 ,  4.  ],
       [10.58,  1.  ],
       [11.08,  2.  ],
       [11.58,  3.  ],
       [12.06,  4.  ],
       [12.54,  1.  ],
       [13.02,  2.  ],
       [13.52,  3.  ]])

In [17]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
# 022dccf = - 1

In [65]:
offset = -1
# Fix the beat numbers
for index in range(len(beats)):
    beats[index][1] = int((index + offset) % 8)
beats

array([[ 0.28,  7.  ],
       [ 0.76,  0.  ],
       [ 1.26,  1.  ],
       [ 1.76,  2.  ],
       [ 2.26,  3.  ],
       [ 2.76,  4.  ],
       [ 3.22,  5.  ],
       [ 3.7 ,  6.  ],
       [ 4.2 ,  7.  ],
       [ 4.68,  0.  ],
       [ 5.16,  1.  ],
       [ 5.66,  2.  ],
       [ 6.16,  3.  ],
       [ 6.64,  4.  ],
       [ 7.14,  5.  ],
       [ 7.64,  6.  ],
       [ 8.12,  7.  ],
       [ 8.62,  0.  ],
       [ 9.1 ,  1.  ],
       [ 9.6 ,  2.  ],
       [10.1 ,  3.  ],
       [10.58,  4.  ],
       [11.08,  5.  ],
       [11.58,  6.  ],
       [12.06,  7.  ],
       [12.54,  0.  ],
       [13.02,  1.  ],
       [13.52,  2.  ]])

In [9]:
'''
beat_start = 0.66
beat_end = 8.2
num_beats = 16
beat_delta = (beat_end - beat_start) / (num_beats - 1)
beats = [[round(beat_start + x * beat_delta, 2), x % 8] for x in range(num_beats)]
beats
'''

'\nbeat_start = 0.66\nbeat_end = 8.2\nnum_beats = 16\nbeat_delta = (beat_end - beat_start) / (num_beats - 1)\nbeats = [[round(beat_start + x * beat_delta, 2), x % 8] for x in range(num_beats)]\nbeats\n'

In [11]:
'''
# Fix the beat numbers
for index in range(12, 16):
    beats[index][1] = beats[index][1] - 4
beats
'''

'\n# Fix the beat numbers\nfor index in range(12, 16):\n    beats[index][1] = beats[index][1] - 4\nbeats\n'

In [67]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{int(x[1] + 1)}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf62.3.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 4363, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc62.11.100 libx264'}}], 'input_number': 0}], 'duration': 13.68, 'bitrate': 4499, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 128, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_n

MoviePy - Done.
MoviePy - Writing video processed/videos/5365862_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/5365862_counts.mp4


In [69]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': [],
        'offset': offset
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1
2
3


In [71]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file

    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8 or (count == 4 and index < len(beats) - 2 and beats[index + 1][1] == 0):
        section += 1

8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1
2
3


In [75]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    True,
    False,
    True,
    True,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    True,
    False,
    False,
    True,
    True,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
]
#leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
5365862_section_1_count_8.jpg
(2, 17, 2)
35394.047
[[593.241   307.18048]
 [592.6665  300.57886]
 [590.68494 298.962  ]
 [566.0147  299.20087]
 [573.41003 296.58356]
 [554.59503 330.07526]
 [559.7685  332.8719 ]
 [595.88715 362.6201 ]
 [598.0909  376.53857]
 [634.12573 360.07062]
 [646.3887  371.07144]
 [519.5581  438.8829 ]
 [541.25183 439.96307]
 [525.0932  522.65784]
 [590.2046  515.4044 ]
 [486.02832 593.8303 ]
 [562.977   600.27844]]
5365862_section_2_count_1.jpg
(2, 17, 2)
36727.797
[[735.7089  318.93536]
 [742.75366 311.53818]
 [730.97455 312.39542]
 [758.7892  310.61957]
 [727.74646 312.3335 ]
 [758.98193 355.91556]
 [733.02893 343.02304]
 [730.5942  406.57834]
 [701.37555 329.99603]
 [690.44934 401.9157 ]
 [690.26874 298.229  ]
 [755.0892  445.7392 ]
 [730.81635 442.65088]
 [705.35187 517.88403]
 [698.18604 516.47125]
 [740.6307  598.0009 ]
 [740.47

In [77]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [79]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [81]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '-',
    '~2',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '~4',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '~4',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)